In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="")  # replace with your Pinecone API key
index_name = "movie-recommender"

# Use free-tier supported region
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")  # ✅ supported
    )

index = pc.Index(index_name)


In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Sample movie descriptions
data = {
    "title": [
        "Inception", "Interstellar", "The Matrix", "The Prestige", "Arrival",
        "The Social Network", "Parasite", "The Dark Knight", "Joker", "Her",
        "Whiplash", "La La Land", "The Imitation Game", "The Theory of Everything", "Ex Machina",
        "The Grand Budapest Hotel", "1917", "Ford v Ferrari", "The Revenant", "Blade Runner 2049",
        "Everything Everywhere All At Once", "Tenet", "Dune", "Oppenheimer", "The Truman Show"
    ],
    "description": [
        "A thief who enters people's dreams and steals secrets from their subconscious.",
        "A team of explorers travel through a wormhole in space to ensure humanity's survival.",
        "A hacker discovers the reality he lives in is a simulation and joins a rebellion.",
        "Two magicians in a bitter rivalry use dangerous illusions to outperform each other.",
        "A linguist tries to communicate with alien visitors before tensions escalate.",
        "The rise of Facebook and the legal battles that followed its creation.",
        "A poor family infiltrates a wealthy household through deception, until things spiral.",
        "A vigilante in Gotham faces chaos when a criminal mastermind tests his limits.",
        "A failed comedian's descent into madness becomes a movement in Gotham.",
        "A lonely man falls in love with an intelligent operating system.",
        "A young drummer faces abuse from a ruthless instructor to achieve greatness.",
        "An aspiring actress and a jazz pianist fall in and out of love in LA.",
        "Mathematician Alan Turing helps crack Nazi codes during World War II.",
        "The life and love of Stephen Hawking as he battles ALS.",
        "A young programmer is selected to test a humanoid robot's AI capabilities.",
        "The misadventures of a concierge at a luxurious European hotel.",
        "Two British soldiers race against time to deliver a message in WWI.",
        "Rival car makers battle for victory at Le Mans in the 1960s.",
        "A frontiersman fights to survive after being mauled by a bear.",
        "A blade runner unearths a secret that could destabilize society.",
        "A woman is swept into a chaotic, multiverse-hopping journey of self-discovery.",
        "A secret agent manipulates time to prevent global catastrophe.",
        "A noble family becomes embroiled in a war for control of a desert planet.",
        "The story of J. Robert Oppenheimer and the making of the atomic bomb.",
        "A man unknowingly lives his entire life inside a reality TV show."
    ]
}


df = pd.DataFrame(data)

model = SentenceTransformer('all-MiniLM-L6-v2')

df["embedding"] = df["description"].apply(lambda x: model.encode(x).tolist())


In [ ]:
vectors = [
    (str(i), row["embedding"], {"title": row["title"]})
    for i, row in df.iterrows()
]

index.upsert(vectors=vectors)

{'upserted_count': 25}

In [ ]:
query_text = "A sci-fi movie about space and survival"
query_vector = model.encode(query_text).tolist()

# Search in Pinecone
results = index.query(vector=query_vector, top_k=3, include_metadata=True)

# Display results
print("Top Matches for:", query_text)
for match in results["matches"]:
    print(f"{match['score']:.2f} - {match['metadata'].get('title', 'N/A')}")

Top Matches for: A sci-fi movie about space and survival
0.45 - Interstellar
0.30 - Dune
0.27 - The Theory of Everything
